In [ ]:
# Set environment variable to avoid symbolic tracing issues
import os
os.environ['TIMM_FUSED_ATTN'] = '0'
import warnings
warnings.filterwarnings('ignore')
from torchvision import transforms
from datasets import load_dataset
from pytorch_grad_cam import run_dff_on_image, GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import cv2
import torch
from typing import List, Callable, Optional

# Import XAI methods
from baselines.GradCAM_v2 import PytorchCAM
from baselines.RISE import RISEBatch
from baselines.Attention import Attention_Map
from baselines.ViT_explanation_generator import LRP, Baselines
from pytorch_grad_cam import GradCAM, ScoreCAM

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from PIL import Image
from pathlib import Path


In [2]:
from transformers import (
    ViTImageProcessor, ViTForImageClassification,
    AutoImageProcessor, EfficientNetForImageClassification,
    ResNetForImageClassification, AutoModel
)
import models_vit as models
from util.datasets import TransformWrapper

#get model
def get_model(task,model,input_size,nb_classes):
    if 'ADCon' in task:
        id2label = {0: "control", 1: "ad"}
        label2id = {v: k for k, v in id2label.items()}
    else:
        id2label = {i: f"class_{i}" for i in range(nb_classes)}
        label2id = {v: k for k, v in id2label.items()}
    processor = None
    if 'RETFound_mae' in model:
        model = models.__dict__['RETFound_mae'](
        img_size=input_size,
        num_classes=nb_classes,
        drop_path_rate=0.2,
        global_pool=True,
    )
    elif 'vit-base-patch16-224' in model:
            # ViT-base-patch16-224 preprocessor
            model_ = 'google/vit-base-patch16-224'
            processor = TransformWrapper(ViTImageProcessor.from_pretrained(model_))
            model = ViTForImageClassification.from_pretrained(
                model_,
                image_size=input_size, #Not in tianhao code, default 224
                num_labels=nb_classes,
                hidden_dropout_prob=0.0, #Not in tianhao code, default 0.0
                attention_probs_dropout_prob=0.0, #Not in tianhao code, default 0.0
                id2label=id2label,
                label2id=label2id,
                ignore_mismatched_sizes=True
            )
    elif 'efficientnet-b4' in model:
        # EfficientNet-B4 preprocessor
        model_ = 'google/efficientnet-b4'
        processor = TransformWrapper(AutoImageProcessor.from_pretrained(model_))
        model = EfficientNetForImageClassification.from_pretrained(
            model_,
            image_size=input_size,
            num_labels=nb_classes,
            dropout_rate=0.0,
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )
    elif 'ResNet-50' in model:
        model_name = 'microsoft/resnet-50'
        processor = TransformWrapper(AutoImageProcessor.from_pretrained(model_name))
        model = ResNetForImageClassification.from_pretrained(
            model_name,
            num_labels=nb_classes,
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )

    return model, processor

In [3]:
# task and dataset
Task_list = ['ADCon','DME']
dataset_fname = 'sampled_labels01.csv'
dataset_dir = '/blue/ruogu.fang/tienyuchang/OCT_EDA'
img_p_fmt = "label_%d/%s" #label index and oct_img name

# model
Model_root = "/blue/ruogu.fang/tienyuchang/RETFound_MAE/output_dir"
Model_fname = "checkpoint-best.pth"
Model_list = ['ResNet-50', 'efficientnet-b4', 'vit-base-patch16-224', 'RETFound_mae']
ADCon_finetuned = [
    "",
    "",
    "",
    "ad_control_detect_data-IRB2024v5_ADCON_DL_data-all-RETFound_mae-OCT-defaulteval---bal_sampler-/"
]
DME_finetuned = [
    "",
    "",
    "DME_binary_all_split-IRB2024_v5-all-google/vit-base-patch16-224-in21k-OCT-bs16ep50lr5e-4optadamw-defaulteval-trsub0--/",
    "DME_binary_all_split-IRB2024_v5-all-RETFound_mae_natureOCT-OCT-bs16ep50lr5e-4optadamw-roc_auceval-trsub0--/"
]

In [ ]:
# Data loading and preprocessing functions
def load_sample_data(task):
    """Load sample images for a given task"""
    df = pd.read_csv(os.path.join(dataset_dir, "%s_sampled"%task, dataset_fname))
    task_df = df[df['label'].isin([0, 1])]  # Adjust based on actual DME labels
    # Sample random images
    sample_df = task_df
    
    images = []
    labels = []
    
    for _, row in sample_df.iterrows():
        # Extract just the filename from oct_img
        filename = os.path.basename(row['OCT']) if isinstance(row['OCT'], str) else row['OCT']
        img_path = os.path.join(dataset_dir, "%s_sampled"%task, img_p_fmt % (row['label'], filename))
        if os.path.exists(img_path):
            try:
                img = Image.open(img_path).convert('RGB')
                images.append(img)
                labels.append(row['label'])
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                continue
    
    return images, labels

def preprocess_image(image, processor=None, input_size=224):
    """Preprocess image for model input"""
    if processor is not None:
        # Check if processor is a TransformWrapper (from util.datasets)
        if hasattr(processor, 'transform'):
            # TransformWrapper expects single image input
            pixel_values = processor(image)
            return pixel_values.unsqueeze(0)  # Add batch dimension
        else:
            # Standard HuggingFace processor
            try:
                inputs = processor(images=image, return_tensors="pt")
            except TypeError:
                # Some processors expect a list
                inputs = processor(images=[image], return_tensors="pt")
            return inputs['pixel_values']
    else:
        # Default preprocessing for RETFound
        transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        return transform(image).unsqueeze(0)

#test dataset
dme_imgs, dme_labels = load_sample_data('DME')
print(dme_imgs[0])
print(dme_labels[0])

<PIL.Image.Image image mode=RGB size=512x496 at 0x1506DE9A7B20>
0


In [5]:

# Load trained models function
def load_trained_model(task, model_name, input_size=224, nb_classes=2):
    """Load a trained model for a specific task"""
    model, processor = get_model(task, model_name, input_size, nb_classes)
    
    # Load model weights based on task and model
    if task == 'ADCon':
        model_paths = ADCon_finetuned
    elif task == 'DME':
        model_paths = DME_finetuned
    else:
        print(f"Unknown task: {task}")
        model.eval()
        return model, processor
    
    model_idx = Model_list.index(model_name)
    model_dir = model_paths[model_idx]
    model_path = os.path.join(Model_root, model_dir, Model_fname)
    
    # Load finetuned model if specified (following main_XAI_evaluation.py pattern)
    if model_path and model_path != '':
        if os.path.exists(model_path):
            try:
                # Load checkpoint
                if model_path.startswith('https'):
                    checkpoint = torch.hub.load_state_dict_from_url(
                        model_path, map_location='cpu', check_hash=True)
                else:
                    checkpoint = torch.load(model_path, map_location='cpu', weights_only=False)
                
                # Extract model state dict
                if 'model' in checkpoint:
                    checkpoint_model = checkpoint['model']
                else:
                    checkpoint_model = checkpoint
                
                # Load with strict=False to handle potential mismatches
                model.load_state_dict(checkpoint_model, strict=False)
                print(f"Resume checkpoint {model_path} for {model_name} on {task}")
                
            except Exception as e:
                print(f"Error loading model {model_name} for {task}: {e}")
                print("Using pretrained weights instead")
        else:
            print(f"Model path not found: {model_path}")
            print(f"Using pretrained weights for {model_name} on {task}")
    else:
        print(f"No checkpoint specified for {model_name} on {task}, using pretrained weights")
    
    model.eval()
    return model, processor

#test
model, processor = load_trained_model('DME', 'RETFound_mae', input_size=224, nb_classes=2)
#print(model)
#print(processor)

Resume checkpoint /blue/ruogu.fang/tienyuchang/RETFound_MAE/output_dir/DME_binary_all_split-IRB2024_v5-all-RETFound_mae_natureOCT-OCT-bs16ep50lr5e-4optadamw-roc_auceval-trsub0--/checkpoint-best.pth for RETFound_mae on DME


In [ ]:
# XAI Methods Implementation
class XAIGenerator:
    def __init__(self, model, model_name, input_size=224):
        self.model = model
        self.model_name = model_name
        self.input_size = input_size
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
        # Initialize XAI methods
        self.init_xai_methods()
    
    def get_model_specific_config(self):
        """Get model-specific configuration for XAI methods"""
        config = {
            'patch_size': 14,
            'gpu_batch': 50,
            'attention_layers': 12
        }
        
        # Model-specific configurations
        if 'resnet' in self.model_name.lower():
            config.update({
                'patch_size': 7,  # ResNet has different spatial resolution
                'gpu_batch': 100,  # ResNet can handle larger batches
                'target_layer_hint': 'layer4'  # ResNet's final conv layer
            })
        elif 'efficientnet' in self.model_name.lower():
            config.update({
                'patch_size': 7,  # EfficientNet spatial resolution
                'gpu_batch': 75,
                'target_layer_hint': 'features'  # EfficientNet's feature extractor
            })
        elif 'vit' in self.model_name.lower():
            config.update({
                'patch_size': 16,  # ViT patch size
                'gpu_batch': 50,
                'attention_layers': 12,  # Standard ViT-Base layers
                'target_layer_hint': 'encoder'  # ViT encoder layers
            })
        elif 'retfound' in self.model_name.lower():
            config.update({
                'patch_size': 16,  # RETFound uses ViT architecture
                'gpu_batch': 50,
                'attention_layers': 12,
                'target_layer_hint': 'blocks'  # RETFound blocks
            })
        
        return config
    
    def init_xai_methods(self):
        """Initialize all XAI methods with model-specific configurations"""
        config = self.get_model_specific_config()
        
        try:
            # GradCAM with model-specific config
            self.gradcam = PytorchCAM(
                self.model, 
                self.model_name, 
                self.input_size, 
                patch_size=config['patch_size'],
                method=GradCAM
            )
            print(f"✓ GradCAM initialized for {self.model_name} (patch_size: {config['patch_size']})")
        except Exception as e:
            print(f"✗ Failed to initialize GradCAM for {self.model_name}: {e}")
            print(self.model)
            self.gradcam = None
        
        try:
            # ScoreCAM with model-specific config
            self.scorecam = PytorchCAM(
                self.model, 
                self.model_name, 
                self.input_size, 
                patch_size=config['patch_size'],
                method=ScoreCAM
            )
            print(f"✓ ScoreCAM initialized for {self.model_name} (patch_size: {config['patch_size']})")
        except Exception as e:
            print(f"✗ Failed to initialize ScoreCAM for {self.model_name}: {e}")
            self.scorecam = None
        
        try:
            # RISE with model-specific batch size
            # Reduce batch for memory-heavy models
            rise_batch = config['gpu_batch']
            if 'retfound' in self.model_name.lower():
                rise_batch = min(rise_batch, 16)
            self.rise = RISEBatch(
                self.model, 
                input_size=(self.input_size, self.input_size), 
                gpu_batch=rise_batch
            )
            print(f"✓ RISE initialized for {self.model_name} (gpu_batch: {rise_batch})")
        except Exception as e:
            print(f"✗ Failed to initialize RISE for {self.model_name}: {e}")
            self.rise = None
        
        try:
            # Attention Maps (only for ViT-based models)
            if 'vit' in self.model_name.lower() or 'retfound' in self.model_name.lower():
                self.attention = Attention_Map(
                    self.model, 
                    self.model_name, 
                    self.input_size, 
                    N=config['attention_layers'],
                    use_rollout=True,
                    print_layers=False  # Disable layer printing to avoid issues
                )
                print(f"✓ Attention initialized for {self.model_name} (layers: {config['attention_layers']})")
            else:
                self.attention = None
                print(f"⚠ Attention skipped for {self.model_name} (not a transformer model)")
        except Exception as e:
            print(f"✗ Failed to initialize Attention for {self.model_name}: {e}")
            print(f"  Note: This may be due to symbolic tracing issues with the model architecture")
            
            # Try alternative attention extraction for RETFound
            if 'retfound' in self.model_name.lower():
                try:
                    print(f"  🔄 Trying alternative attention extraction for RETFound...")
                    self.attention = self._create_simple_attention_extractor()
                    print(f"✓ Alternative attention method initialized for {self.model_name}")
                except Exception as e2:
                    print(f"✗ Alternative attention method also failed: {e2}")
                    self.attention = None
            else:
                self.attention = None
        
        # LRP (requires special model implementation)
        self.lrp = None  # Will implement if model supports it
    
    def generate_gradcam(self, image_tensor, target_class=None):
        """Generate GradCAM heatmap"""
        if self.gradcam is None:
            return None
        
        try:
            image_tensor = image_tensor.to(self.device)
            
            if target_class is None:
                # Get predicted class
                with torch.no_grad():
                    outputs = self.model(image_tensor)
                    target_class = outputs.argmax(dim=1).item()
            
            targets = [ClassifierOutputTarget(target_class)]
            heatmap = self.gradcam(image_tensor, targets)
            return heatmap[0] if len(heatmap) > 0 else None
        except Exception as e:
            print(f"Error generating GradCAM: {e}")
            return None
    
    def generate_scorecam(self, image_tensor, target_class=None):
        """Generate ScoreCAM heatmap"""
        if self.scorecam is None:
            return None
        
        try:
            image_tensor = image_tensor.to(self.device)
            
            if target_class is None:
                # Get predicted class
                with torch.no_grad():
                    outputs = self.model(image_tensor)
                    target_class = outputs.argmax(dim=1).item()
            
            targets = [ClassifierOutputTarget(target_class)]
            heatmap = self.scorecam(image_tensor, targets)
            return heatmap[0] if len(heatmap) > 0 else None
        except Exception as e:
            print(f"Error generating ScoreCAM: {e}")
            return None
    
    def generate_rise(self, image_tensor, target_class=None):
        """Generate RISE heatmap"""
        if self.rise is None:
            return None
        
        try:
            image_tensor = image_tensor.to(self.device)
            
            if target_class is None:
                # Get predicted class
                with torch.no_grad():
                    outputs = self.model(image_tensor)
                    target_class = outputs.argmax(dim=1).item()
            
            heatmaps = self.rise(image_tensor)
            return heatmaps[0, target_class] if heatmaps is not None else None
        except Exception as e:
            print(f"Error generating RISE: {e}")
            return None
    
    def generate_attention(self, image_tensor):
        """Generate Attention heatmap"""
        if self.attention is None:
            return None
        
        try:
            image_tensor = image_tensor.to(self.device)
            attention_map = self.attention(image_tensor)
            return attention_map[0] if attention_map is not None else None
        except Exception as e:
            print(f"Error generating Attention: {e}")
            return None
    
    def generate_all_heatmaps(self, image_tensor, target_class=None):
        """Generate all available heatmaps for an image"""
        heatmaps = {}
        
        # GradCAM
        gradcam_map = self.generate_gradcam(image_tensor, target_class)
        if gradcam_map is not None:
            heatmaps['GradCAM'] = gradcam_map
        
        # ScoreCAM
        scorecam_map = self.generate_scorecam(image_tensor, target_class)
        if scorecam_map is not None:
            heatmaps['ScoreCAM'] = scorecam_map
        
        # RISE
        rise_map = self.generate_rise(image_tensor, target_class)
        if rise_map is not None:
            heatmaps['RISE'] = rise_map
        
        # Attention
        attention_map = self.generate_attention(image_tensor)
        if attention_map is not None:
            heatmaps['Attention'] = attention_map
        
        return heatmaps
    
    def _create_simple_attention_extractor(self):
        """Create a simple attention extractor that doesn't use symbolic tracing"""
        class SimpleAttentionExtractor:
            def __init__(self, model, model_name, input_size):
                self.model = model
                self.model_name = model_name
                self.input_size = input_size
                self.attention_maps = []
                
                # Register hooks to capture attention weights
                self.hooks = []
                if hasattr(model, 'blocks'):
                    for i, block in enumerate(model.blocks):
                        if hasattr(block, 'attn'):
                            hook = block.attn.register_forward_hook(self._attention_hook)
                            self.hooks.append(hook)
            
            def _attention_hook(self, module, input, output):
                """Hook to capture attention weights"""
                # Try to get attention weights from different possible attributes
                if hasattr(module, 'attention_probs'):
                    self.attention_maps.append(module.attention_probs.detach())
                elif hasattr(module, 'attn_weights'):
                    self.attention_maps.append(module.attn_weights.detach())
                elif len(output) > 1 and isinstance(output[1], torch.Tensor):
                    # Some models return attention as second output
                    self.attention_maps.append(output[1].detach())
            
            def __call__(self, x):
                """Extract attention maps"""
                self.attention_maps = []
                
                # Forward pass to trigger hooks
                with torch.no_grad():
                    _ = self.model(x)
                
                if len(self.attention_maps) > 0:
                    try:
                        # Use the last attention layer
                        last_attention = self.attention_maps[-1]  # Shape: (B, num_heads, num_tokens, num_tokens)
                        
                        # Average over heads and extract CLS attention to patches
                        if len(last_attention.shape) == 4:  # (B, num_heads, num_tokens, num_tokens)
                            cls_attention = last_attention.mean(dim=1)[:, 0, 1:]  # (B, num_patches)
                        else:
                            # Fallback for different attention shapes
                            cls_attention = last_attention[:, 0, 1:] if last_attention.shape[1] > 1 else last_attention[:, 0]
                        
                        # Reshape to spatial dimensions
                        B = cls_attention.shape[0]
                        num_patches = cls_attention.shape[1]
                        patch_size = int(num_patches ** 0.5)
                        
                        if patch_size * patch_size == num_patches:
                            attention_maps = cls_attention.reshape(B, patch_size, patch_size)
                            
                            # Resize to input size
                            attention_maps_resized = torch.nn.functional.interpolate(
                                attention_maps.unsqueeze(1), 
                                size=(self.input_size, self.input_size), 
                                mode='bilinear', 
                                align_corners=False
                            ).squeeze(1)
                            
                            return attention_maps_resized.cpu().numpy()
                    except Exception as e:
                        print(f"Error processing attention maps: {e}")
                
                # Fallback: return zeros
                return np.zeros((x.shape[0], self.input_size, self.input_size))
            
            def cleanup(self):
                """Remove hooks"""
                for hook in self.hooks:
                    hook.remove()
        
        return SimpleAttentionExtractor(self.model, self.model_name, self.input_size)
#test
for model_name in Model_list:
    model, processor = load_trained_model('DME', model_name, 224)
    XAIGenerator(model, model_name)

✓ GradCAM initialized for ResNet-50 (patch_size: 7)
✓ ScoreCAM initialized for ResNet-50 (patch_size: 7)
Masks are loaded.
✓ RISE initialized for ResNet-50 (gpu_batch: 100)
⚠ Attention skipped for ResNet-50 (not a transformer model)

📋 Target Layer Information for ResNet-50:
  ⚠ Could not determine exact target layer: 'VisionTransformer' object has no attribute 'resnet'
  🎯 CAM Methods: Using auto-resolved target layer
✓ GradCAM initialized for efficientnet-b4 (patch_size: 7)
✓ ScoreCAM initialized for efficientnet-b4 (patch_size: 7)
Masks are loaded.
✓ RISE initialized for efficientnet-b4 (gpu_batch: 75)
⚠ Attention skipped for efficientnet-b4 (not a transformer model)

📋 Target Layer Information for efficientnet-b4:
  🎯 CAM Methods: Auto-resolved target layer
  🔍 RISE: Uses entire model with masking
  👁 Attention: Not applicable (non-transformer model)
✓ GradCAM initialized for vit-base-patch16-224 (patch_size: 16)
✓ ScoreCAM initialized for vit-base-patch16-224 (patch_size: 16)
Mask

In [7]:
# Visualization functions
def normalize_heatmap(heatmap):
    """Normalize heatmap to 0-1 range"""
    if heatmap is None:
        return None
    
    heatmap = np.array(heatmap)
    if heatmap.max() == heatmap.min():
        return np.zeros_like(heatmap)
    
    return (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

def overlay_heatmap_on_image(image, heatmap, alpha=0.4, colormap='jet'):
    """Overlay heatmap on original image"""
    if heatmap is None:
        return np.array(image)
    
    # Normalize heatmap
    heatmap_norm = normalize_heatmap(heatmap)
    
    # Resize heatmap to match image size
    if isinstance(image, Image.Image):
        image_array = np.array(image)
        image_size = image.size
    else:
        image_array = image
        image_size = (image.shape[1], image.shape[0])
    
    heatmap_resized = cv2.resize(heatmap_norm, image_size)
    
    # Apply colormap
    cmap = plt.get_cmap(colormap)
    heatmap_colored = cmap(heatmap_resized)[:, :, :3]  # Remove alpha channel
    
    # Normalize image
    image_norm = image_array.astype(np.float32) / 255.0
    
    # Overlay
    overlay = alpha * heatmap_colored + (1 - alpha) * image_norm
    overlay = np.clip(overlay, 0, 1)
    
    return (overlay * 255).astype(np.uint8)

def create_heatmap_grid(results_dict, save_path=None):
    """Create a comprehensive grid visualization of all heatmaps"""
    tasks = list(results_dict.keys())
    models = Model_list
    xai_methods = ['GradCAM', 'ScoreCAM', 'RISE', 'Attention']
    
    # Calculate grid dimensions
    n_tasks = len(tasks)
    n_models = len(models)
    n_methods = len(xai_methods)
    n_samples = len(results_dict[tasks[0]][models[0]]['images']) if tasks and models else 0
    
    # Create figure
    fig_width = max(20, n_methods * 4)
    fig_height = max(15, n_tasks * n_models * 3)
    fig, axes = plt.subplots(
        n_tasks * n_models * n_samples, 
        n_methods + 1,  # +1 for original image
        figsize=(fig_width, fig_height)
    )
    
    if n_tasks * n_models * n_samples == 1:
        axes = axes.reshape(1, -1)
    elif len(axes.shape) == 1:
        axes = axes.reshape(-1, 1)
    
    row_idx = 0
    
    for task_idx, task in enumerate(tasks):
        for model_idx, model in enumerate(models):
            if model not in results_dict[task]:
                continue
                
            model_results = results_dict[task][model]
            images = model_results['images']
            labels = model_results['labels']
            heatmaps = model_results['heatmaps']
            
            for sample_idx in range(len(images)):
                image = images[sample_idx]
                label = labels[sample_idx]
                sample_heatmaps = heatmaps[sample_idx]
                
                # Original image
                axes[row_idx, 0].imshow(image)
                axes[row_idx, 0].set_title(f'{task}-{model}\nSample {sample_idx+1} (Label: {label})')
                axes[row_idx, 0].axis('off')
                
                # Heatmaps
                for method_idx, method in enumerate(xai_methods):
                    col_idx = method_idx + 1
                    
                    if method in sample_heatmaps and sample_heatmaps[method] is not None:
                        overlay = overlay_heatmap_on_image(image, sample_heatmaps[method])
                        axes[row_idx, col_idx].imshow(overlay)
                        axes[row_idx, col_idx].set_title(f'{method}')
                    else:
                        axes[row_idx, col_idx].text(0.5, 0.5, 'N/A', 
                                                  ha='center', va='center', 
                                                  transform=axes[row_idx, col_idx].transAxes)
                        axes[row_idx, col_idx].set_title(f'{method}')
                    
                    axes[row_idx, col_idx].axis('off')
                
                row_idx += 1
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Heatmap grid saved to {save_path}")
    
    plt.show()
    
    return fig


In [9]:
# Main execution: Generate heatmaps for all combinations
def generate_comprehensive_heatmaps(num_samples=3):
    """Generate heatmaps for all task-model combinations"""
    
    results = {}
    input_size = 224
    
    print("Starting comprehensive heatmap generation...")
    print(f"Tasks: {Task_list}")
    print(f"Models: {Model_list}")
    print(f"Samples per task: {num_samples}")
    
    for task in Task_list:
        print(f"\n=== Processing Task: {task} ===")
        results[task] = {}
        
        # Load sample data for this task
        try:
            images, labels = load_sample_data(task)
            print(f"Loaded {len(images)} images for {task}")
        except Exception as e:
            print(f"Error loading data for {task}: {e}")
            continue
        
        for model_name in Model_list:
            print(f"\n--- Processing Model: {model_name} ---")
            
            try:
                # Load trained model
                model, processor = load_trained_model(task, model_name, input_size)
                
                # Initialize XAI generator
                xai_generator = XAIGenerator(model, model_name, input_size)
                
                # Store results for this model
                results[task][model_name] = {
                    'images': images,
                    'labels': labels,
                    'heatmaps': []
                }
                
                # Process each image
                for idx, (image, label) in enumerate(zip(images, labels)):
                    print(f"Processing image {idx+1}/{len(images)} (Label: {label})")
                    
                    # Preprocess image
                    image_tensor = preprocess_image(image, processor, input_size)
                    
                    # Generate all heatmaps for this image
                    heatmaps = xai_generator.generate_all_heatmaps(image_tensor, target_class=label)
                    
                    results[task][model_name]['heatmaps'].append(heatmaps)
                    
                    print(f"Generated heatmaps: {list(heatmaps.keys())}")
                
                print(f"Completed {model_name} for {task}")
                
            except Exception as e:
                print(f"Error processing {model_name} for {task}: {e}")
                results[task][model_name] = {
                    'images': images,
                    'labels': labels,
                    'heatmaps': [{}] * len(images)
                }
                continue
    
    return results

# Print model-specific configuration summary
def print_model_config_summary():
    """Print a summary of model-specific configurations"""
    print("\n📊 MODEL-SPECIFIC CONFIGURATION SUMMARY")
    print("=" * 80)
    
    configs = []
    for model_name in Model_list:
        # Create temporary XAI generator to get config
        dummy_model = torch.nn.Identity()  # Placeholder
        temp_generator = XAIGenerator.__new__(XAIGenerator)
        temp_generator.model_name = model_name
        config = temp_generator.get_model_specific_config()
        
        configs.append({
            'Model': model_name,
            'Patch Size': config['patch_size'],
            'GPU Batch': config['gpu_batch'],
            'Attention Layers': config.get('attention_layers', 'N/A'),
            'Target Hint': config.get('target_layer_hint', 'Auto-resolve')
        })
    
    # Print as table
    print(f"{'Model':<20} {'Patch Size':<12} {'GPU Batch':<12} {'Att. Layers':<12} {'Target Hint':<15}")
    print("-" * 80)
    for config in configs:
        print(f"{config['Model']:<20} {config['Patch Size']:<12} {config['GPU Batch']:<12} {config['Attention Layers']:<12} {config['Target Hint']:<15}")
    
    print("\nConfiguration Details:")
    print("• Patch Size: Spatial resolution for CAM methods")
    print("• GPU Batch: Batch size for RISE method")
    print("• Att. Layers: Number of attention layers for ViT models")
    print("• Target Hint: Architecture-specific target layer guidance")

print_model_config_summary()

# Execute the heatmap generation
print("\n=== Starting Heatmap Generation ===")
heatmap_results = generate_comprehensive_heatmaps(num_samples=2)  # Start with 2 samples per task



📊 MODEL-SPECIFIC CONFIGURATION SUMMARY
Model                Patch Size   GPU Batch    Att. Layers  Target Hint    
--------------------------------------------------------------------------------
ResNet-50            7            100          12           layer4         
efficientnet-b4      7            75           12           features       
vit-base-patch16-224 16           50           12           encoder        
RETFound_mae         16           50           12           blocks         

Configuration Details:
• Patch Size: Spatial resolution for CAM methods
• GPU Batch: Batch size for RISE method
• Att. Layers: Number of attention layers for ViT models
• Target Hint: Architecture-specific target layer guidance

=== Starting Heatmap Generation ===
Starting comprehensive heatmap generation...
Tasks: ['ADCon', 'DME']
Models: ['ResNet-50', 'efficientnet-b4', 'vit-base-patch16-224', 'RETFound_mae']
Samples per task: 2

=== Processing Task: ADCon ===
Loaded 200 images for ADCon

--

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

...

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b4 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1792]) in the checkpoint and torch.Size([2, 1792]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model path not found: /blue/ruogu.fang/tienyuchang/RETFound_MAE/output_dir/checkpoint-best.pth
Using pretrained weights for efficientnet-b4 on DME
✗ Failed to initialize GradCAM for efficientnet-b4: Unsupported model for GradCAM: cannot resolve target layer automatically.
✗ Failed to initialize ScoreCAM for efficientnet-b4: Unsupported model for GradCAM: cannot resolve target layer automatically.
Masks are loaded.
✓ RISE initialized for efficientnet-b4 (gpu_batch: 75)
⚠ Attention skipped for efficientnet-b4 (not a transformer model)

📋 Target Layer Information for efficientnet-b4:
  🎯 CAM Methods: model.efficientnet.encoder.blocks[-1] (Final EfficientNet block)
  🔍 RISE: Uses entire model with masking
  👁 Attention: Not applicable (non-transformer model)
Processing image 1/200 (Label: 0)
Error processing efficientnet-b4 for DME: __call__() got an unexpected keyword argument 'images'

--- Processing Model: vit-base-patch16-224 ---


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Resume checkpoint /blue/ruogu.fang/tienyuchang/RETFound_MAE/output_dir/DME_binary_all_split-IRB2024_v5-all-google/vit-base-patch16-224-in21k-OCT-bs16ep50lr5e-4optadamw-defaulteval-trsub0--/checkpoint-best.pth for vit-base-patch16-224 on DME
✓ GradCAM initialized for vit-base-patch16-224 (patch_size: 16)
✓ ScoreCAM initialized for vit-base-patch16-224 (patch_size: 16)
Masks are loaded.
✓ RISE initialized for vit-base-patch16-224 (gpu_batch: 50)
✓ Attention initialized for vit-base-patch16-224 (layers: 12)

📋 Target Layer Information for vit-base-patch16-224:
  🎯 CAM Methods: model.vit.encoder.layer[-1] (Final encoder layer)
  🔍 RISE: Uses entire model with masking
  👁 Attention: Uses all transformer layers with rollout
Processing image 1/200 (Label: 0)
Error processing vit-base-patch16-224 for DME: __call__() got an unexpected keyword argument 'images'

--- Processing Model: RETFound_mae ---
Resume checkpoint /blue/ruogu.fang/tienyuchang/RETFound_MAE/output_dir/DME_binary_all_split-IRB2

In [ ]:
# Create comprehensive visualization
print("\n=== Creating Comprehensive Visualization ===")

# Create the main heatmap grid
output_dir = Path("./heatmap_results")
output_dir.mkdir(exist_ok=True)

main_grid_path = output_dir / "comprehensive_heatmap_grid.png"
fig = create_heatmap_grid(heatmap_results, save_path=main_grid_path)

print(f"Main visualization saved to: {main_grid_path}")


In [ ]:
# Create individual task-specific visualizations
def create_task_specific_grids(results_dict, output_dir):
    """Create separate visualizations for each task"""
    
    for task in results_dict.keys():
        print(f"Creating visualization for {task}...")
        
        # Create task-specific results dict
        task_results = {task: results_dict[task]}
        
        # Create visualization
        task_grid_path = output_dir / f"{task}_heatmap_grid.png"
        fig = create_heatmap_grid(task_results, save_path=task_grid_path)
        
        plt.close(fig)  # Close to save memory
        print(f"{task} visualization saved to: {task_grid_path}")

# Generate task-specific grids
create_task_specific_grids(heatmap_results, output_dir)

print("\n=== Heatmap Generation Complete ===")
print(f"All results saved in: {output_dir}")
print("Generated files:")
for file_path in output_dir.glob("*.png"):
    print(f"  - {file_path.name}")


In [ ]:
# Summary and Analysis
def print_heatmap_summary(results_dict):
    """Print a summary of generated heatmaps"""
    
    print("\n=== HEATMAP GENERATION SUMMARY ===")
    
    total_combinations = 0
    successful_combinations = 0
    method_success_count = {'GradCAM': 0, 'ScoreCAM': 0, 'RISE': 0, 'Attention': 0}
    
    for task in results_dict.keys():
        print(f"\nTask: {task}")
        
        for model in results_dict[task].keys():
            model_results = results_dict[task][model]
            n_samples = len(model_results['images'])
            
            print(f"  Model: {model} ({n_samples} samples)")
            
            # Count successful heatmaps for each method
            for sample_idx, heatmaps in enumerate(model_results['heatmaps']):
                total_combinations += 1
                
                if heatmaps:  # If any heatmaps were generated
                    successful_combinations += 1
                
                for method in ['GradCAM', 'ScoreCAM', 'RISE', 'Attention']:
                    if method in heatmaps and heatmaps[method] is not None:
                        method_success_count[method] += 1
                        print(f"    Sample {sample_idx+1}: {method} ✓")
                    else:
                        print(f"    Sample {sample_idx+1}: {method} ✗")
    
    print(f"\n=== OVERALL STATISTICS ===")
    print(f"Total task-model-sample combinations: {total_combinations}")
    print(f"Successful combinations: {successful_combinations}")
    print(f"Success rate: {successful_combinations/total_combinations*100:.1f}%")
    
    print(f"\n=== METHOD SUCCESS RATES ===")
    for method, count in method_success_count.items():
        success_rate = count / total_combinations * 100 if total_combinations > 0 else 0
        print(f"{method}: {count}/{total_combinations} ({success_rate:.1f}%)")

# Print summary
print_heatmap_summary(heatmap_results)
